In [1]:
#
# This code comes from: https://www.kaggle.com/code/hojjatk/read-mnist-dataset
#
import matplotlib
matplotlib.use('TkAgg')
import numpy as np  # linear algebra
import struct
from array import array
from os.path import join
import random
import matplotlib.pyplot as plt


#
# MNIST Data Loader Class
#
class MnistDataloader(object):
    def __init__(self, training_images_filepath, training_labels_filepath,
                 test_images_filepath, test_labels_filepath):
        self.training_images_filepath = training_images_filepath
        self.training_labels_filepath = training_labels_filepath
        self.test_images_filepath = test_images_filepath
        self.test_labels_filepath = test_labels_filepath

    def read_images_labels(self, images_filepath, labels_filepath):
        labels = []
        with open(labels_filepath, 'rb') as file:
            magic, size = struct.unpack(">II", file.read(8))
            if magic != 2049:
                raise ValueError('Magic number mismatch, expected 2049, got {}'.format(magic))
            labels = array("B", file.read())

        with open(images_filepath, 'rb') as file:
            magic, size, rows, cols = struct.unpack(">IIII", file.read(16))
            if magic != 2051:
                raise ValueError('Magic number mismatch, expected 2051, got {}'.format(magic))
            image_data = array("B", file.read())
        images = []
        for i in range(size):
            images.append([0] * rows * cols)
        for i in range(size):
            img = np.array(image_data[i * rows * cols:(i + 1) * rows * cols])
            img = img.reshape(28, 28)
            images[i][:] = img

        return images, labels

    def load_data(self):
        x_train, y_train = self.read_images_labels(self.training_images_filepath, self.training_labels_filepath)
        x_test, y_test = self.read_images_labels(self.test_images_filepath, self.test_labels_filepath)
        return (np.array(x_train), np.array(y_train)),(np.array(x_test), np.array(y_test))

#
# Set file paths based on added MNIST Datasets
#
input_path = '/Users/jmoore/Documents/ML/archive'
training_images_filepath = join(input_path, 'train-images.idx3-ubyte')
training_labels_filepath = join(input_path, 'train-labels.idx1-ubyte')
test_images_filepath = join(input_path, 't10k-images.idx3-ubyte')
test_labels_filepath = join(input_path, 't10k-labels.idx1-ubyte')

#
# Helper function to show a list of images with their relating titles
#
def show_images(images, title_texts):
    cols = 5
    rows = int(len(images)/cols) + 1
    plt.figure(figsize=(30,20))
    index = 1
    for x in zip(images, title_texts):
        image = x[0]
        title_text = x[1]
        plt.subplot(rows, cols, index)
        plt.imshow(image, cmap=plt.cm.gray)
        if (title_text != ''):
            plt.title(title_text, fontsize=15)
        index += 1
    plt.show()

#
# Load MINST dataset
#
mnist_dataloader = MnistDataloader(training_images_filepath, training_labels_filepath, test_images_filepath, test_labels_filepath)
(x_train_init, y_train_init), (x_test_init, y_test_init) = mnist_dataloader.load_data()


np.save(join(input_path, 'mnist-train-x.npy'), x_train_init.reshape(len(x_train_init), 784))
np.save(join(input_path, 'mnist-train-y.npy'), y_train_init)
np.save(join(input_path, 'mnist-test-x.npy'), x_test_init.reshape(len(x_test_init), 784))
np.save(join(input_path, 'mnist-test-y.npy'), y_test_init)

#
# Show some random training and test images
#
images_2_show = []
titles_2_show = []
for i in range(0, 10):
    r = random.randint(1, 60000)
    images_2_show.append(x_train_init[r])
    titles_2_show.append('training image [' + str(r) + '] = ' + str(y_train_init[r]))

for i in range(0, 5):
    r = random.randint(1, 10000)
    images_2_show.append(x_test_init[r])
    titles_2_show.append('test image [' + str(r) + '] = ' + str(y_test_init[r]))

show_images(images_2_show, titles_2_show)

In [5]:
from sklearn.model_selection import train_test_split
from MLP import MultilayerPerceptron, Layer, Mish, Softplus, CrossEntropy, Relu, Softmax, SquaredError

def one_hot_encode(y: np.ndarray, num_classes: int) -> np.ndarray:
    encoded = np.zeros((y.shape[0], num_classes))
    for i, label in enumerate(y):
        encoded[i, int(label)] = 1
    return encoded

# Flatten images and scale pixel values to [0,1]
x_train_flat = x_train_init.reshape(x_train_init.shape[0], -1) / 255.0
x_test_flat = x_test_init.reshape(x_test_init.shape[0], -1) / 255.0

# Convert labels to one-hot encoding (10 classes)
y_train_onehot = one_hot_encode(y_train_init, 10)
y_test_onehot = one_hot_encode(y_test_init, 10)

 # Split the original training set into 80% training and 20% validation
x_train, x_val, y_train, y_val = train_test_split(x_train_flat, y_train_onehot, test_size=0.2, random_state=42, shuffle=True)

mlp = MultilayerPerceptron([
    # Input layer: 784 features -> Hidden layer with 128 neurons (ReLU) with dropout 20%
    Layer(fan_in=784, fan_out=256, activation_function=Relu(), dropout_rate=0.2),
    # Hidden layer: 128 -> 64 neurons (ReLU) with dropout 20%
    Layer(fan_in=256, fan_out=128, activation_function=Relu(), dropout_rate=0.2),
    # Output layer: 64 -> 10 neurons with Softmax activation (for 10 classes)
    Layer(fan_in=128, fan_out=10, activation_function=Softmax())
])


# Train the MLP

training_losses, validation_losses = mlp.train(
    train_x=x_train,
    train_y=y_train,
    val_x=x_val,
    val_y=y_val,
    loss_func=CrossEntropy(),
    learning_rate=1e-3,
    batch_size=32,
    epochs=32,
    use_rmsprop=True,
    decay_rate=0.9,
    epsilon=1e-8
)


# Evaluate the Model on the Test Set and Report Accuracy

y_test_pred = np.argmax(mlp.forward(x_test_flat, training=False), axis=1)
# y_test contains the true labels (as integers)
test_accuracy = np.mean(y_test_pred == np.array(y_test_init))
print(f"Test Accuracy: {test_accuracy * 100}%")


plt.plot(training_losses, label="Training Loss")
plt.plot(validation_losses, label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("MNIST Training and Validation Loss")
plt.legend()
plt.show()

Epoch 1  :::  Train Loss=0.32285033555506165  :::  Val Loss=0.19786256182618198
Epoch 2  :::  Train Loss=0.17133560511403373  :::  Val Loss=0.15756030233568524
Epoch 3  :::  Train Loss=0.14505447309507105  :::  Val Loss=0.19035578663443184
Epoch 4  :::  Train Loss=0.13460801643849196  :::  Val Loss=0.19163586276245018
Epoch 5  :::  Train Loss=0.13659909560182767  :::  Val Loss=0.1810042131074192
Epoch 6  :::  Train Loss=0.13922604486537735  :::  Val Loss=0.1918326876673106
Epoch 7  :::  Train Loss=0.1309579631098805  :::  Val Loss=0.2084296153638575
Epoch 8  :::  Train Loss=0.13621430282837657  :::  Val Loss=0.2230608281953006
Epoch 9  :::  Train Loss=0.13352388357379555  :::  Val Loss=0.260152428183953
Epoch 10  :::  Train Loss=0.12984692332134615  :::  Val Loss=0.22091328896487267
Epoch 11  :::  Train Loss=0.12834361796983246  :::  Val Loss=0.25618463722244755
Epoch 12  :::  Train Loss=0.13426228758586697  :::  Val Loss=0.23959321400433414
Epoch 13  :::  Train Loss=0.1333487287069731

: 

In [4]:
# Assume:
# - x_test_flat is your test data of shape (n_samples, 784) normalized to [0,1]
# - y_test_init contains the true labels as integers
# - mlp is your trained MultilayerPerceptron

# Select one sample per class based on the true labels
selected_indices = {}
for idx, true_label in enumerate(y_test_init):
    if true_label not in selected_indices:
        selected_indices[true_label] = idx
    if len(selected_indices) == 10:
        break

# Convert the selected indices to a sorted list (by digit)
selected_indices = [selected_indices[digit] for digit in sorted(selected_indices.keys())]

# Get the corresponding test images and reshape them to 28x28 for display
selected_images = x_test_flat[selected_indices].reshape(-1, 28, 28)

# Get the predictions for the selected samples
selected_preds = np.argmax(mlp.forward(x_test_flat[selected_indices], training=False), axis=1)

# Plot the images with predicted labels
import matplotlib.pyplot as plt
cols = 5
rows = 2
plt.figure(figsize=(12, 6))
for i, idx in enumerate(selected_indices):
    plt.subplot(rows, cols, i+1)
    plt.imshow(selected_images[i], cmap='gray')
    plt.title(f"Pred: {selected_preds[i]}")
    plt.axis('off')
plt.tight_layout()
plt.show()
